<a href="https://colab.research.google.com/github/UNKNOWNhacking/CS3491-ARTIFICIAL-INTELLIGENCE-AND-MACHINE-LEARNING/blob/main/CS3491_ARTIFICIAL_INTELLIGENCE_AND_MACHINE_LEARNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **EXERCISE 1 - BFS**

In [2]:
from collections import deque

def bfs(graph, start_node):
    # Create a queue for BFS
    queue = deque([start_node])
    # Set to keep track of visited nodes
    visited = set([start_node])

    while queue:
        # Dequeue a node from the queue
        node = queue.popleft()
        print(node, end=" ")

        # Get all adjacent vertices of the dequeued node
        for neighbor in graph[node]:
            if neighbor not in visited:
                # Mark neighbor as visited and enqueue it
                visited.add(neighbor)
                queue.append(neighbor)

# Example usage:
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

print("Breadth-First Search:")
bfs(graph, 'A')


Breadth-First Search:
A B C D E F 

# **EXERCISE 1 - DFS**

In [4]:
def dfs_recursive(graph, node, visited=None):
    if visited is None:
        visited = set()

    # Mark the node as visited
    visited.add(node)
    print(node, end=" ")

    # Recur for all the vertices adjacent to this node
    for neighbor in graph[node]:
        if neighbor not in visited:
            dfs_recursive(graph, neighbor, visited)

# Example graph definition
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

# Example usage:
print("\n\nDepth-First Search (Recursive):")
dfs_recursive(graph, 'A')




Depth-First Search (Recursive):
A B D E F C 

# **EXERCISE 2 - A* Search Algorithm**

In [5]:
import heapq

def a_star(graph, start, goal, h):
    # Priority queue for A* (min-heap)
    open_list = []
    heapq.heappush(open_list, (0, start))  # (f_score, node)

    # Dictionaries to store the actual cost and parent of each node
    g_score = {node: float('inf') for node in graph}
    g_score[start] = 0
    came_from = {}

    while open_list:
        # Get the node with the lowest f_score
        current_f, current = heapq.heappop(open_list)

        if current == goal:
            # Reconstruct and return the path
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]  # Return reversed path

        # Explore neighbors
        for neighbor, cost in graph[current]:
            tentative_g_score = g_score[current] + cost

            if tentative_g_score < g_score[neighbor]:
                # Update the best known path and g_score for neighbor
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score = tentative_g_score + h(neighbor, goal)
                heapq.heappush(open_list, (f_score, neighbor))

    return None  # If no path is found

# Heuristic function (Manhattan distance for simplicity)
def heuristic(node, goal):
    x1, y1 = node
    x2, y2 = goal
    return abs(x1 - x2) + abs(y1 - y2)

# Example graph as an adjacency list
graph = {
    (0, 0): [((1, 0), 1), ((0, 1), 1)],
    (1, 0): [((0, 0), 1), ((1, 1), 1)],
    (0, 1): [((0, 0), 1), ((1, 1), 1)],
    (1, 1): [((1, 0), 1), ((0, 1), 1), ((2, 2), 1)],
    (2, 2): []
}

# Example usage
start = (0, 0)
goal = (2, 2)
print("A* Search Path:", a_star(graph, start, goal, heuristic))


A* Search Path: [(0, 0), (0, 1), (1, 1), (2, 2)]


# **EXERCISE 2 - Memory-Bounded A* (MA*)**

In [6]:
import heapq

class Node:
    def __init__(self, state, g, h, parent=None):
        self.state = state
        self.g = g  # Cost from start to node
        self.h = h  # Heuristic estimate
        self.f = g + h  # Estimated total cost
        self.parent = parent  # To reconstruct the path

    def __lt__(self, other):
        return self.f < other.f

def memory_bounded_a_star(graph, start, goal, h, memory_limit):
    # Priority queue (open list) for nodes to explore
    open_list = []
    heapq.heappush(open_list, Node(start, 0, h(start, goal)))

    best_solution = None  # Track the best path

    while open_list:
        # Trim open_list if memory limit is exceeded
        if len(open_list) > memory_limit:
            open_list = heapq.nsmallest(memory_limit, open_list)  # Keep the best nodes

        # Get the node with the lowest f-score
        current = heapq.heappop(open_list)

        if current.state == goal:
            # Goal reached: reconstruct the path
            path = []
            while current:
                path.append(current.state)
                current = current.parent
            return path[::-1]  # Return reversed path (from start to goal)

        # Explore neighbors
        for neighbor, cost in graph[current.state]:
            g = current.g + cost  # Update the g-score (actual cost to the neighbor)
            f = g + h(neighbor, goal)  # Total estimated cost f(n) = g(n) + h(n)
            heapq.heappush(open_list, Node(neighbor, g, h(neighbor, goal), current))

        # Track the best solution so far based on f-score
        if not best_solution or current.f < best_solution.f:
            best_solution = current

    # If no solution is found, return the best solution found so far
    return None if best_solution is None else reconstruct_path(best_solution)

def reconstruct_path(node):
    """Reconstruct the path from start to goal by following parent nodes."""
    path = []
    while node:
        path.append(node.state)
        node = node.parent
    return path[::-1]  # Return the path from start to goal

# Heuristic function (Manhattan distance for this example)
def heuristic(node, goal):
    x1, y1 = node
    x2, y2 = goal
    return abs(x1 - x2) + abs(y1 - y2)

# Example graph as an adjacency list
graph = {
    (0, 0): [((1, 0), 1), ((0, 1), 1)],
    (1, 0): [((0, 0), 1), ((1, 1), 1)],
    (0, 1): [((0, 0), 1), ((1, 1), 1)],
    (1, 1): [((1, 0), 1), ((0, 1), 1), ((2, 2), 1)],
    (2, 2): []
}

# Example usage with memory limit
start = (0, 0)
goal = (2, 2)
memory_limit = 5

print("Memory-Bounded A* Search Path:", memory_bounded_a_star(graph, start, goal, heuristic, memory_limit))


Memory-Bounded A* Search Path: [(0, 0), (1, 0), (1, 1), (2, 2)]


# **EXERCISE 3 - Gaussian Naive Bayes (For continuous data)**

In [7]:
import numpy as np

class GaussianNaiveBayes:
    def fit(self, X, y):
        # Calculate means, variances and priors for each class
        self.classes = np.unique(y)
        self.mean = {}
        self.variance = {}
        self.priors = {}

        for c in self.classes:
            X_c = X[y == c]
            self.mean[c] = np.mean(X_c, axis=0)
            self.variance[c] = np.var(X_c, axis=0)
            self.priors[c] = X_c.shape[0] / X.shape[0]

    def _gaussian_pdf(self, class_idx, x):
        mean = self.mean[class_idx]
        var = self.variance[class_idx]
        numerator = np.exp(-((x - mean) ** 2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator

    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return np.array(predictions)

    def _predict(self, x):
        posteriors = []

        for c in self.classes:
            prior = np.log(self.priors[c])  # log(P(c))
            class_conditional = np.sum(np.log(self._gaussian_pdf(c, x)))  # log(P(x|c))
            posterior = prior + class_conditional
            posteriors.append(posterior)

        return self.classes[np.argmax(posteriors)]


# Example usage:
# Create some sample data for 2 classes
X = np.array([[1.0, 2.1], [1.1, 1.9], [3.1, 2.9], [3.0, 3.2], [4.0, 4.5], [5.0, 5.0]])
y = np.array([0, 0, 1, 1, 1, 1])

gnb = GaussianNaiveBayes()
gnb.fit(X, y)
predictions = gnb.predict(X)
print("Predictions:", predictions)


Predictions: [0 0 1 1 1 1]


<ipython-input-7-6c289d028cc7>:33: RuntimeWarning: divide by zero encountered in log
  class_conditional = np.sum(np.log(self._gaussian_pdf(c, x)))  # log(P(x|c))


# **EXERCISE 3 - Multinomial Naive Bayes (For discrete data)**

In [8]:
import numpy as np

class MultinomialNaiveBayes:
    def fit(self, X, y):
        # Calculate priors and likelihoods for each class
        self.classes = np.unique(y)
        self.class_count = len(self.classes)
        self.feature_count = X.shape[1]

        # Likelihood (P(word|class)) and priors P(class)
        self.likelihood = np.zeros((self.class_count, self.feature_count))
        self.priors = np.zeros(self.class_count)

        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.likelihood[idx, :] = np.sum(X_c, axis=0) + 1  # Laplace smoothing
            self.priors[idx] = X_c.shape[0] / float(X.shape[0])

        # Normalize likelihood to represent probabilities
        self.likelihood = self.likelihood / np.sum(self.likelihood, axis=1, keepdims=True)

    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return np.array(predictions)

    def _predict(self, x):
        posteriors = []

        for idx, c in enumerate(self.classes):
            prior = np.log(self.priors[idx])
            class_likelihood = np.sum(np.log(self.likelihood[idx, :]) * x)
            posterior = prior + class_likelihood
            posteriors.append(posterior)

        return self.classes[np.argmax(posteriors)]


# Example usage with text data
# Columns are words in a vocabulary, rows are document word counts
X = np.array([[3, 2, 0], [1, 1, 0], [0, 0, 5], [0, 1, 4]])
y = np.array([0, 0, 1, 1])  # Two classes (e.g., spam vs non-spam)

mnb = MultinomialNaiveBayes()
mnb.fit(X, y)
predictions = mnb.predict(X)
print("Predictions:", predictions)


Predictions: [0 0 1 1]


# **EXERCISE 4**

In [ ]:
#First Install This Package
!pip install pgmpy
#It take 5 mintes to install

In [11]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Step 1: Define the structure of the Bayesian Network
model = BayesianNetwork([('Rain', 'Grass'), ('Sprinkler', 'Grass')])

# Step 2: Define the Conditional Probability Distributions (CPDs)

# P(Rain) - Prior probability for Rain
cpd_rain = TabularCPD(variable='Rain', variable_card=2, values=[[0.7], [0.3]])

# P(Sprinkler) - Prior probability for Sprinkler
cpd_sprinkler = TabularCPD(variable='Sprinkler', variable_card=2, values=[[0.6], [0.4]])

# P(Grass | Rain, Sprinkler) - Conditional probability table for Grass
cpd_grass = TabularCPD(
    variable='Grass',
    variable_card=2,
    values=[[0.99, 0.9, 0.8, 0.0],  # P(Grass=False)
            [0.01, 0.1, 0.2, 1.0]],  # P(Grass=True)
    evidence=['Rain', 'Sprinkler'],
    evidence_card=[2, 2]
)

# Step 3: Add CPDs to the model
model.add_cpds(cpd_rain, cpd_sprinkler, cpd_grass)

# Verify the model is correct (CPDs should sum to 1)
assert model.check_model()

# Step 4: Perform inference
inference = VariableElimination(model)

# Query 1: What is the probability that the Grass is wet (True) given that it rained?
result_1 = inference.query(variables=['Grass'], evidence={'Rain': 1})
print("P(Grass=True | Rain=True):\n", result_1)

# Query 2: What is the probability of rain given that the Grass is wet and the sprinkler is on?
result_2 = inference.query(variables=['Rain'], evidence={'Grass': 1, 'Sprinkler': 1})
print("P(Rain | Grass=True, Sprinkler=True):\n", result_2)


P(Grass=True | Rain=True):
 +----------+--------------+
| Grass    |   phi(Grass) |
+==========+==============+
| Grass(0) |       0.4800 |
+----------+--------------+
| Grass(1) |       0.5200 |
+----------+--------------+
P(Rain | Grass=True, Sprinkler=True):
 +---------+-------------+
| Rain    |   phi(Rain) |
+=========+=============+
| Rain(0) |      0.1892 |
+---------+-------------+
| Rain(1) |      0.8108 |
+---------+-------------+
